## Этапы (простой) обработки текста

<img src="images/textm.png">


## Декодирование


**Def.**  
перевод последовательности байт в последовательность символов

* Распаковка  
*plain/.zip/.gz/...*
* Кодировка  
*ASCII/utf-8/Windows-1251/...*
* Формат  
*csv/xml/json/doc...*

Кроме того: что такое документ?



## Разбиение на токены
**Def.**  
разбиение последовательности символов на части (токены), возможно, исключая из рассмотрения некоторые символы  
Наивный подход: разделить строку пробелами и выкинуть знаки препинания  


*Трисия любила Нью-Йорк, поскольку любовь к Нью-Йорку могла положительно повлиять на ее карьеру.*  


**Проблемы:**  
* example@example.com, 127.0.0.1
* С++, C#
* York University vs New York University
* Зависимость от языка (“Lebensversicherungsgesellschaftsangestellter”, “l’amour”)
Альтернатива: n-граммы

In [3]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kirillkuznecov/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
from nltk.tokenize import RegexpTokenizer


s = "Трисия любила Нью-Йорк, поскольку любовь к Нью-Йорку могла положительно повлиять на ее карьеру."

tokenizer = RegexpTokenizer("\w+|[^\w\s]+")
for t in tokenizer.tokenize(s): 
    print(t)

Трисия
любила
Нью
-
Йорк
,
поскольку
любовь
к
Нью
-
Йорку
могла
положительно
повлиять
на
ее
карьеру
.


## Стоп-слова
**Def.**  
Наиболее частые слова в языке, не содержащие никакой информации о содержании текста



In [ ]:
from nltk.corpus import stopwords


print(" ".join(stopwords.words("russian")[1:20]))

Проблема: “To be or not to be"

## Нормализация
**Def.**  
Приведение токенов к единому виду для того, чтобы избавиться от поверхностной разницы в написании  

Подходы  
* сформулировать набор правил, по которым преобразуется токен  
Нью-Йорк → нью-йорк → ньюйорк → ньюиорк
* явно хранить связи между токенами (WordNet – Princeton)  
машина → автомобиль, Windows 6→ window

In [ ]:
s = "Нью-Йорк"
s1 = s.lower()
print(s1)

In [ ]:
import re
s2 = re.sub(r"\W", "", s1, flags=re.U)
print(s2)

In [ ]:
s3 = re.sub(r"й", u"и", s2, flags=re.U)
print(s3)

## Стемминг и Лемматизация
**Def.**  
Приведение грамматических форм слова и однокоренных слов к единой основе (lemma):
* Stemming – с помощью простых эвристических правил
  * Porter (Cambridge – 1980)
        5 этапов, на каждом применяется набор правил, таких как
            sses → ss (caresses → caress)
            ies → i (ponies → poni)

  * Lovins (1968)
  * Paice (1990)
  * другие
* Lemmatization – с использованием словарей и морфологического анализа


## Стемминг

In [ ]:
from nltk.stem.snowball import PorterStemmer
from nltk.stem.snowball import RussianStemmer


s = PorterStemmer()
print(s.stem("Tokenization"))
print(s.stem("stemming"))

r = RussianStemmer()
print(r.stem("Авиация"))
print(r.stem("национальный"))

**Наблюдение**  
для сложных языков лучше подходит лемматизация

## Лемматизация

In [ ]:
# !pip install pymorphy2

In [ ]:
import pymorphy2


morph = pymorphy2.MorphAnalyzer()
print(morph.parse("думающему")[0].normal_form)

## Heaps' law
Эмпирическая закономерность в лингвистике, описывающая распределение числа уникальных слов в документе (или наборе документов) как функцию от его длины.

$$
M = k T^\beta, \;M \text{ -- размер словаря}, \; T \text{ -- количество слов в корпусе}
$$
$$
30 \leq k \leq 100, \; b \approx 0.5
$$

<img src="images/dim.png">
<img src="images/heaps.png">

## Представление документов
**Boolean Model.** Присутствие или отсутствие слова в документе  
**Bag of Words.** Порядок токенов не важен  

*Погода была ужасная, принцесса была прекрасная.
Или все было наоборот?*

Координаты
* Мультиномиальные: количество токенов в документе
* Числовые: взвешенное количество токенов в документе

## Zipf's law
Эмпирическая закономерность распределения частоты слов естественного языка

$t_1, \ldots, t_N$ - токены, отранжированные по убыванию частоты
   	
$f_1, \dots, f_N$ - соответствующие частоты

**Закон Ципфа**
	$$
	f_i = \frac{c}{i^k}
	$$	
	
	Что еще? Посещаемость сайтов, количество друзей, население городов...
<img src="images/zipf.png">


In [19]:
# Введите своё решение ниже
import string
import pandas as pd

s = pd.Series(
    ["Авторитет и дружба вода и огонь, вещи разнородные и враждебные; равенство условие дружбы.",
     "Нет силы более могучей, чем знание; человек, вооружённый знанием, — непобедим.",
     "Истинная сила человека не в порывах, а в нерушимом спокойствии.",
     "Величайшее богатство народа его язык."
     ],
    dtype="string"
)

exclude = string.punctuation
s = s.str.replace(f'[{string.punctuation}]', '', regex=True)
new_s = s.str.lower().str.strip(" ").str.split(" ")
new_s

0    [авторитет, и, дружба, вода, и, огонь, вещи, р...
1    [нет, силы, более, могучей, чем, знание, челов...
2    [истинная, сила, человека, не, в, порывах, а, ...
3           [величайшее, богатство, народа, его, язык]
dtype: object

In [8]:
import pandas as pd
import string
import pymorphy2
from nltk.corpus import stopwords

morpher = pymorphy2.MorphAnalyzer()

sw = set(stopwords.words('russian'))



In [9]:
def preprocess_txt(line):
    # Почистим строку от пунктуации. Для этого пробежимся по каждому символу и проверим, не является ли он знаком пунктуации
    exclude = set(string.punctuation)
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    # Лемматизируем все слова в нашем тексте
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

s = pd.Series(
    ["Разруха не в клозетах, а в головах. ",
     "Люди как люди. Любят деньги, но ведь это всегда было.",
     "Там хорошо, где нас нет: в прошлом нас уже нет, и оно кажется прекрасным.",
     "Нет силы более могучей, чем знание; человек, вооружённый знанием, — непобедим."
     ],
    dtype="string"
)

# Применяем функцию для предобработки к Series
s = s.apply(lambda x: preprocess_txt(x))
print(s)

0                            [разруха, клозет, голова]
1              [человек, человек, любить, деньга, это]
2                 [прошлый, оно, казаться, прекрасный]
3    [сила, могучий, знание, человек, вооружённый, ...
dtype: object


In [39]:
## Введите своё решение ниже
import pandas as pd
import string



exclude = string.punctuation

def preprocess_txt(line):
    clean_line = pd.Series(line)
    clean_line = clean_line.str.replace(f'[{exclude}]', '', regex=True)
    clean_line = clean_line.str.lower()
    clean_line = clean_line.str.split().str[::-1]
    
    return clean_line

line = "Авторитет и дружба вода и огонь, вещи разнородные и враждебные; равенство условие дружбы."

preprocess_txt(line)


0    [дружбы, условие, равенство, враждебные, и, ра...
dtype: object

In [40]:
data = pd.DataFrame({
    'label': [
        'В. Белинский',
        'М. Горький',
        'Л. Толстой',
        'М. Шолохов'
        ],
    'text': [
        "Авторитет и дружба вода и огонь, вещи разнородные и враждебные; равенство условие дружбы.",
        "Нет силы более могучей, чем знание; человек, вооружённый знанием, непобедим.",
        "Истинная сила человека не в порывах, а в нерушимом спокойствии.",
        "Величайшее богатство народа его язык."
    ]
})

In [46]:
data['text']=data['text'].apply(preprocess_txt)

In [47]:
data

,label,text
0,В. Белинский,"[дружбы, условие, равенство, враждебные, и, ра..."
1,М. Горький,"[непобедим, знанием, вооружённый, человек, зна..."
2,Л. Толстой,"[спокойствии, нерушимом, в, а, порывах, в, не,..."
3,М. Шолохов,"[язык, его, народа, богатство, величайшее]"
